<b>TUGAS 2<b>
<b>ROHADI<b>

In [1]:
pip install transformers datasets torch

Note: you may need to restart the kernel to use updated packages.


<b>Dataset<b>

In [5]:
file = 'C:/Users/rohad/liputan6_data.tar.gz'
tar = tarfile.open(file, "r:gz")
tar.extractall(path="dataset_directory")
tar.close()

In [7]:
!tar -xzf C:/Users/rohad/liputan6_data.tar.gz

In [8]:
!pip install -q transformers accelerate datasets==2.17.1 evaluate==0.4.1 seqeval

In [10]:
pip install --upgrade jupyter ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -------------------------- ------------- 92.2/139.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ----------------- ---------------------- 92.2/214.4 kB ? eta -:--:--
   ---------------------------------------- 214.4/214.4 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.3 MB 5.5 MB/s eta 0:00:01
   ------- -------------------------------- 0.5/2.3 MB 4.8 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.3 MB 5.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.3 MB 5.4 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 5.5 MB/s eta 0:00:01
   ----------------------- ---------------- 1.4/2.3 MB 5.2 MB/s eta 0:00:01
   -------------------

<b>Processing Dataset<b>

In [11]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset, load_from_disk
from datasets import Dataset, DatasetDict, concatenate_datasets
from transformers import BertTokenizer, BertModel, EncoderDecoderModel, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [12]:
import glob
import json
import re

train_file = glob.glob("liputan6_data/canonical/train/*.json")
train_file.sort(key=lambda f: int(re.sub('\D', '', f)))

eval_file = glob.glob("liputan6_data/canonical/test/*.json")
eval_file.sort(key=lambda f: int(re.sub('\D', '', f)))

train_data = []
eval_data = []

for i in train_file:
  with open(i, "r", encoding="utf-8") as f:
    d = json.load(f)
    train_data.append(d)

for i in eval_file:
  with open(i, "r", encoding="utf-8") as f:
    d = json.load(f)
    eval_data.append(d)

print(f"train data: {len(train_data)}")
print(f"eval data: {len(eval_data)}")

train data: 193883
eval data: 10972


In [13]:
train_data[0].keys()

dict_keys(['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary'])

In [14]:
train_data = train_data[:5000]
eval_data = eval_data[:500]

In [15]:
import numpy as np
import pandas as pd

def custom_join(words):
  result = ' '.join(words)
  result = result.replace("Liputan6 . com", "Liputan6.com")
  result = result.replace(" , ", ", ")
  result = result.replace(" . ", ". ")
  result = result.replace(" ( ", " (")
  result = result.replace(" ) ", ") ")
  return result


def make_dataset_df(data):
  clean_article = []
  clean_summary = []

  for item in data:
    clean_article_sentence = []
    for arr in item['clean_article']:
      clean_article_sentence.extend(arr)
    joined_str1 = custom_join(clean_article_sentence)
    clean_article.append(joined_str1)

    clean_summary_sentence = []
    for arr in item['clean_summary']:
      clean_summary_sentence.extend(arr)
    joined_str2 = custom_join(clean_summary_sentence)
    clean_summary.append(joined_str2)

  df = pd.DataFrame({'clean_article': clean_article, 'clean_summary': clean_summary})
  return df

In [16]:
train_df = make_dataset_df(train_data)
eval_df = make_dataset_df(eval_data)

In [17]:
train_df

,clean_article,clean_summary
0,"Liputan6.com, Ambon : Partai Bulan Bintang wil...",Konflik Ambon telah berlangsung selama tiga ta...
1,"Liputan6.com, Denpasar : Berbeda dengan sebagi...",Masyarakat Bali merayakan Tahun Baru dengan tr...
2,"Liputan6.com, Jakarta : Partai Keadilan bertek...",Partai Keadilan menargetkan tambahan sejuta pe...
3,"Liputan6.com, Jakarta : Sekitar Rumah Makan Ay...",Pascaledakan granat di depan Rumah Makan Ayam ...
4,"Liputan6.com, Jambi : Ratusan hektare sawah di...","Bencana Banjir di Jambi, juga mengakibatkan ra..."
...,...,...
4995,"Liputan6.com, Makassar : Pengadilan Negeri Mak...",Ratusan mahasiswa dan dosen nekat menutup jala...
4996,"Liputan6.com, Medan : Proses pembentukan dua p...",Pemisahan provinsi dianggap hanya akan menamba...
4997,"Liputan6.com, Pekanbaru : Ratusan orang yang t...",Ratusan anggota Forum Masyarakat Lancang Kunin...
4998,"Liputan6.com, Jember : Harga bawang merah di k...","Akibat pasokan membludak, harga bawang merah d..."


In [18]:
eval_df

,clean_article,clean_summary
0,"Liputan6.com, Jakarta : Kepolisian Daerah Riau...",Kapolda Riau baru Brigjen Pol. Johny Yodjana b...
1,"Liputan6.com, Jakarta : Bank Indonesia dinilai...",Kendati Bank Sentral AS menurunkan suku bungan...
2,"Liputan6.com, Jakarta : Berbagai kendala mengh...",Pemerintah bermaksud akan lebih mengandalkan s...
3,"Liputan6.com, Jakarta : Penghapusan beberapa p...","Revisi Kepmennaker Nomor 78 Tahun 2001, dinila..."
4,"Liputan6.com, Jakarta : Operasi Sadar Jaya yan...",Polisi menangkap 32 pengunjung Diskotik Mileni...
...,...,...
495,"Liputan6.com, Jakarta : Ketua Asosiasi Tekstil...",Sejumlah pengusaha tekstil berharap pemerintah...
496,"Liputan6.com, Jember : Sebanyak 394 gerbong ke...",PT Pusri tak lagi menyewa 394 gerbong milik PT...
497,"Liputan6.com, Jakarta : Presiden Abdurrahman W...","Sekali lagi, Presiden Wahid membantah rencana ..."
498,"Liputan6.com, Medan : Arus pengungsi Aceh eks ...",Pemda Sumatra Utara kewalahan menangani pengun...


In [19]:
train_df['clean_article'][5]

'Liputan6.com, Makassar : Kondisi kesehatan Wakil Komandan Batalyon Resimen I Brigade Mobil Ajun Komisaris Polisi Tukijan membaik [ baca : Wadan Brimob Ambon Tertembak ]. Namun dalam pengamatan SCTV, baru-baru ini, Tukijan yang sempat krtitis karena tertembak peluru nyasar di bagian dahi sebelah kanan, masih terbaring lemah. Tukijan yang juga Wakil Komandan Satuan Tugas Operasi Mutiara Empat di Ambon, Maluku, sempat dalam kondisi kritis selama empat hari. Tukijan dievakuasi dari Ambon Jumat silam dan kini dirawat di Rumah Sakit Pelamonia Makassar, Sulawesi Selatan. Menurut dokter di rumah sakit tersebut, korban kini sudah dapat menggerakkan otot motoriknya tanpa bantuan alat elektrik. Kepala Dinas Penerangan TNI Angkatan Darat Brigadir Jenderal Ismet menyatakan, insiden tembak menembak itu terjadi di perbatasan Desa Tantui dan Galala adalah antara Brimob dan Marinir. Sementara menurut Kapolda Sulsel Inspektur Jenderal Polisi Firman Gani, jenis peluru yang bersarang di kepala korban ber

In [20]:
train_df['clean_summary'][5]

'Ajun Komisaris Polisi Tukijan, korban baku tembak di Ambon kondisinya mulai membaik. Peluru nyasar yang menembus dahi korban berasal dari senjata jenis AK .'

In [21]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [22]:
train_dataset

Dataset({
    features: ['clean_article', 'clean_summary'],
    num_rows: 5000
})

In [23]:
eval_dataset

Dataset({
    features: ['clean_article', 'clean_summary'],
    num_rows: 500
})

In [24]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('cahya/bert-base-indonesian-1.5G')

C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# Define the tokenize_data function to accept the tokenizer as a parameter
def tokenize_data(example, tokenizer):
    input_encoding = tokenizer(example['clean_article'], 
                                padding='max_length', 
                                truncation=True, 
                                max_length=512,
                                clean_up_tokenization_spaces=True)
    target_encoding = tokenizer(example['clean_summary'], 
                                padding='max_length', 
                                truncation=True, 
                                max_length=128,
                                clean_up_tokenization_spaces=True)
    return {
        'input_ids': input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encoding['input_ids']
    }

In [26]:
# Map the function with tokenizer as an argument
tokenized_train = train_dataset.map(tokenize_data, batched=True, num_proc=4, fn_kwargs={'tokenizer': tokenizer})
tokenized_eval = eval_dataset.map(tokenize_data, batched=True, num_proc=4, fn_kwargs={'tokenizer': tokenizer})

Map (num_proc=4): 100%|██████████| 500/500 [00:06<00:00, 82.55 examples/s] 


In [27]:
tokenizer.pad_token

'[PAD]'

In [28]:
tokenizer.cls_token

'[CLS]'

In [29]:
# make Bert2Bert (Encoder-Decoder model)
model = EncoderDecoderModel.from_encoder_decoder_pretrained('cahya/bert-base-indonesian-1.5G', 'cahya/bert-base-indonesian-1.5G')

Some weights of BertLMHeadModel were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.

In [33]:
# Define special tokens
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

In [34]:
model.config.encoder.vocab_size

32000

In [35]:
# Set configurations for the encoder and decoder
model.config.encoder.max_length = 512
model.config.decoder.max_length = 128
model.config.decoder.min_length = 12
model.config.length_penalty = 2.0
model.config.early_stopping = True

In [36]:
!pip install --upgrade accelerate

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [39]:
from transformers import TrainingArguments

In [40]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=4,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_strategy='epoch',           # updated parameter name
    save_strategy='epoch',
)

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [42]:
trainer.train()

C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,0.886300,0.964951
2,0.715000,0.913296
3,0.500700,0.905551


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'length_penalty': 2.0}
C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\generation\configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `num_beams` is set to 1

TrainOutput(global_step=1875, training_loss=0.8885666852315267, metrics={'train_runtime': 67746.0156, 'train_samples_per_second': 0.221, 'train_steps_per_second': 0.028, 'total_flos': 9201879613440000.0, 'train_loss': 0.8885666852315267, 'epoch': 3.0})

<b>Evaluasi Data<b>

In [43]:
results = trainer.evaluate(tokenized_eval)

C:\Users\rohad\anaconda3\envs\cuda_test\Lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


In [44]:
print(results)

{'eval_loss': 0.9055507183074951, 'eval_runtime': 354.7589, 'eval_samples_per_second': 1.409, 'eval_steps_per_second': 0.352, 'epoch': 3.0}
